# Get NDVI mean values from Sentinel-2 L2A for all features in a FeatureCollection given a period of time 

In [ ]:
from datetime import datetime
from datetime import date
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.ticker import NullFormatter
from matplotlib.dates import MonthLocator, DateFormatter


import ee
ee.Initialize()

In [ ]:
# Functions
def ndvi (image):
    image = image.clip(plots)
    ndvi = image.normalizedDifference(['B8', 'B4'])
    return ndvi

def mean (image):
    mean = image.reduceRegion(reducer = ee.Reducer.mean(), geometry = geometry)
    return ee.Feature(None, {'mean': mean.get('nd'), 'plot':plot})

In [ ]:
# Parameters
start = "2019-10-01" #sowing date
end = "2020-04-20"   #harvest date
plots = ee.Collection.loadTable("users/user123/file123") #path to file, in this case, it is a shapefile with many geometries 

# Image collection with NDVI images (Sentinel-2 L2A) corresponding to all plots in the period of interest
sentinel = ee.ImageCollection("COPERNICUS/S2_SR")\
            .filterDate(start, end)\
            .filterBounds(plots)\
            .select(['B8', 'B4'])\
            .filterMetadata("CLOUDY_PIXEL_PERCENTAGE", "less_than", 20)\
            .map(ndvi)

# Number of images retrieved
number_of_img = len(sentinel.getInfo()['features'])
number_of_img


In [ ]:
data = []

for plot in plots.getInfo()['features']:
    geometry = plot['geometry']
    name = plot ['properties']['layer'] #the plot name is defined in a column called "layer"
    images_reduced_to_mean_by_region = sentinel.filterBounds(geometry).map(mean)
    for feature in images_reduced_to_mean_by_region.getInfo()['features']:
        try:
            data.append([name, feature['id'], feature['properties']['mean']])
        except: 
            pass
        
            
#number of features
len (data)

In [ ]:
# Prepare data to export as csv

# Export raw data, a csv file with columns plot, date and NDVI
data_ = pd.DataFrame(data)
data_.columns = ['plot', 'date', 'NDVI']
data_['date'] = data_['date'].str[0:4].str.cat(data_['date'].str[4:6], sep='-').str.cat(data_['date'].str[6:8], sep='-')
data_.to_csv(f'raw_data_{datetime.now()}.csv')

# Export data discarding dates with NDVI values less tan 0.1 and group by date in case there are more than one NDVI value that day due to 2 tiles contain the plot
filter_data = data_.loc[lambda data_: data_['NDVI'] > 0.1, :] #discard images with NDVI values less than 0.1, assume the images are cloudy
filter_data = filter_data.groupby(['plot', 'date']).mean()
filter_data = filter_data.reset_index()
filter_data.to_csv(f'filter_data_{datetime.now()}.csv')